# 表格型数据集

{mod}`pyarrow.dataset` 模块提供了高效处理表格型数据集的功能，这些数据集可能比内存大，并包含多个文件。这包括：
- 支持不同来源和文件格式以及不同文件系统（本地、云端）的统一接口。
- 源的发现（爬取目录，处理基于目录的分区数据集，基本模式规范化等）
- 优化读取，具有谓词下推（过滤行）、投影（选择和派生列），以及可选的并行读取功能。

目前支持的文件格式包括 Parquet、Feather/Arrow IPC、CSV 和 ORC（请注意，目前只能读取 ORC 数据集，尚不支持写入）。未来的目标是扩展对其他文件格式和数据源（例如数据库连接）的支持。